# Производительность объектно-реляционной модели на примере работы с Postgres

<p> https://en.wikipedia.org/wiki/PostgreSQL
<p> http://initd.org/psycopg/


```sql
CREATE TABLE "region" (
    id SERIAL PRIMARY KEY,
    name VARCHAR(64) NOT NULL);

CREATE TABLE "group" (
    id SERIAL PRIMARY KEY,
    name VARCHAR(256) NOT NULL);

CREATE TABLE "crimestatsocial" (
    id SERIAL PRIMARY KEY,
    data json NOT NULL,
    reg_id INTEGER NOT NULL,
    group_id INTEGER NOT NULL,
    FOREIGN KEY (reg_id)
        REFERENCES "region" (id)
        ON UPDATE CASCADE ON DELETE CASCADE,
    FOREIGN KEY (group_id)
        REFERENCES "group" (id)
        ON UPDATE CASCADE ON DELETE CASCADE);
```
```js
data = {
    "year": <number>,
    "category": <string>,
    "gender": <string>,
    "value": <number>,
}
```

In [1]:
import sys
import time

import psycopg2

sys.path.append('../config/')
from config import config

In [2]:
params = config(section="obj_postgres")
conn = psycopg2.connect(**params)
cur = conn.cursor()
cur.execute("""SELECT version();""")
cur.fetchall()[0][0][:17]

'PostgreSQL 9.5.12'

In [3]:
commands = (
    """
    SELECT
        region.id AS reg_id,
        region.name AS reg_name,
        crimestatsocial.data->'year' AS year,
        "group".id AS group_id,
        "group".name AS group_name,
        crimestatsocial.data->'category' AS category,
        crimestatsocial.data->'gender' AS gender,
        crimestatsocial.data->'value' AS value
    FROM crimestatsocial, region, "group"
    WHERE crimestatsocial.group_id = "group".id
    AND crimestatsocial.reg_id = region.id;""",

    """
    SELECT row_to_json(row) FROM (
        SELECT
            region.id AS reg_id,
            region.name AS reg_name,
            crimestatsocial.data->'year' AS year,
            "group".id AS group_id,
            "group".name AS group_name,
            crimestatsocial.data->'category' AS category,
            crimestatsocial.data->'gender' AS gender,
            crimestatsocial.data->'value' AS value
        FROM crimestatsocial, region, "group"
        WHERE crimestatsocial.group_id = "group".id
        AND crimestatsocial.reg_id = region.id
    ) row;""",

    """
    SELECT
        region.id AS reg_id,
        region.name AS reg_name,
        crimestatsocial.data->'year' AS year,
        "group".id AS group_id,
        "group".name AS group_name,
        crimestatsocial.data->'category' AS category,
        crimestatsocial.data->'gender' AS gender,
        crimestatsocial.data->'value' AS value
    FROM crimestatsocial, region, "group"
    WHERE crimestatsocial.group_id = "group".id
    AND crimestatsocial.reg_id = region.id
    AND CAST(crimestatsocial.data->>'year' AS integer)=2016;""",

    """
    SELECT row_to_json(row) FROM (
        SELECT
            region.id AS reg_id,
            region.name AS reg_name,
            crimestatsocial.data->'year' AS year,
            "group".id AS group_id,
            "group".name AS group_name,
            crimestatsocial.data->'category' AS category,
            crimestatsocial.data->'gender' AS gender,
            crimestatsocial.data->'value' AS value
        FROM crimestatsocial, region, "group"
        WHERE crimestatsocial.group_id = "group".id
        AND crimestatsocial.reg_id = region.id
        AND CAST(crimestatsocial.data->>'year' AS integer)=2016
    ) row;""",

    """
    SELECT group_id, COUNT("group_id")
    FROM "crimestatsocial"
    WHERE CAST(crimestatsocial.data->>'year' AS integer)=2016
    GROUP BY "group_id";""",

    """
    SELECT crimestatsocial.data->>'year', COUNT(*)
    FROM "crimestatsocial"
    GROUP BY crimestatsocial.data->>'year';""",

    """
    SELECT COUNT("id") AS count
    FROM "group";""",

    """
    INSERT INTO
        "crimestatsocial"
        (id, data, reg_id, group_id)
    VALUES (%s, %s, %s, %s);""",

    """
    UPDATE "crimestatsocial"
    SET data = '{"year": 2017, "category": "14-15", "gender": "Мужчины", "value": 1}'
    WHERE "id" = %s;""",

    """
    DELETE FROM "crimestatsocial"
    WHERE "id" = %s;""",
)

## 1. Собираются исходные кортежи

In [4]:
cur.execute(commands[0])
fetchall = cur.fetchall()
n_rows = len(fetchall)
print(n_rows)
print(fetchall[0])

27384
(1100, 'Российская Федерация', 2016, 14002, 'по возрасту', '14-15', 'Мужчины', 13573)


In [5]:
%timeit cur.execute(commands[0])

993 ms ± 88.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit cur.execute(commands[1])

1.48 s ± 115 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## 2. Собираются исходные кортежи за 2016 год

In [7]:
cur.execute(commands[2])
fetchall = cur.fetchall()
print(len(fetchall))
print(fetchall[0])

3948
(1100, 'Российская Федерация', 2016, 14002, 'по возрасту', '14-15', 'Мужчины', 13573)


In [8]:
%timeit cur.execute(commands[2])

335 ms ± 27.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%timeit cur.execute(commands[3])

405 ms ± 44 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## 3. Подсчет исходных кортежей за 2016 год по группам

In [10]:
cur.execute(commands[4])
cur.fetchall()

[(14002, 1128), (14007, 376), (14006, 1692), (14005, 752)]

In [11]:
%timeit cur.execute(commands[4])

167 ms ± 1.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## 4. Подсчет исходных кортежей по годам

In [12]:
cur.execute(commands[5])
cur.fetchall()

[('2011', 3864),
 ('2010', 3864),
 ('2014', 3990),
 ('2015', 3990),
 ('2016', 3948),
 ('2012', 3864),
 ('2013', 3864)]

In [13]:
%timeit cur.execute(commands[5])

201 ms ± 20.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## 5. Подсчет уникальных групп

In [14]:
def fun():
    cur.execute(commands[6])
    fetchall = cur.fetchall()
    return fetchall[0][0]

fun()

4

In [15]:
%timeit fun()

817 µs ± 68.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## 6. Вставка кортежа

In [16]:
json = '{"year": 2018, "category": "14-15", "gender": "Мужчины", "value": 1234567890}'
item = (n_rows + 1, json, 1100, 14002, )
cur.execute(commands[7], item)
conn.commit()

In [17]:
i = int(1e7)
def fun():
    global i
    cur.execute(commands[7], (i, json, 1100, 14002, ))
    i += 1

In [18]:
%timeit -n 1000 -r 5 fun()

1.33 ms ± 159 µs per loop (mean ± std. dev. of 5 runs, 1000 loops each)


In [19]:
conn.commit()

## 7. Изменение кортежа

In [20]:
cur.execute(commands[8], (n_rows + 1, ))
conn.commit()

In [21]:
i = int(1e7)
def fun():
    global i
    cur.execute(commands[8], (i, ))
    i += 1

In [22]:
%timeit -n 1000 -r 5 fun()

1.22 ms ± 20.1 µs per loop (mean ± std. dev. of 5 runs, 1000 loops each)


In [23]:
conn.commit()

## 8. Удаление кортежа

In [24]:
cur.execute(commands[9], (n_rows + 1, ))
conn.commit()

In [25]:
i = int(1e7)
def fun():
    global i
    cur.execute(commands[9], (i, ))
    i += 1

In [26]:
%timeit -n 1000 -r 5 fun()

905 µs ± 44.9 µs per loop (mean ± std. dev. of 5 runs, 1000 loops each)


In [27]:
conn.commit()

In [28]:
cur.close()
conn.close()